In [1]:
import numpy as np
import pandas as pd

# Parsing LUNA’s annotation data

The `candidates.csv` file contains information about all lumps that potentially look like nodules, whether those lumps are malignant, benign tumors, or something else altogether. We’ll use this as the basis for building a complete list of candidates that can then be split into our training and validation datasets.

In [4]:
# Counts the number of lines in the file
! wc -l /storage/dlwpt/candidates.csv

551066 /storage/dlwpt/candidates.csv


In [6]:
# Prints the first few lines of the file
# The first line of the .csv file defines the column headers
! head /storage/dlwpt/candidates.csv

seriesuid,coordX,coordY,coordZ,class
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-56.08,-67.85,-311.92,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,53.21,-244.41,-245.17,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.66,-121.8,-286.62,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-33.66,-72.75,-308.41,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-32.25,-85.36,-362.51,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-26.65,-203.07,-165.07,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-74.99,-114.79,-311.92,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-16.14,-248.61,-239.55,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,135.89,-141.41,-252.2,0


In [8]:
# Counts the number of lines that end (class) with 1, which indicates malignancy
! grep ',1$' /storage/dlwpt/candidates.csv | wc -l

1351


So we have 551,000 lines, each with a seriesuid (which we’ll call series_uid in the code), some (X,Y,Z) coordinates, and a class column that corresponds to the nodule status (it’s a Boolean value: 0 for a candidate that is not an actual nodule, and 1 for a candidate that is a nodule, either malignant or benign). We have 1,351 candidates flagged as actual nodules.

The `annotations.csv` file contains information about some of the candidates that have been flagged as nodules. We are interested in the `diameter_mm` information in particular:

In [10]:
#This is a different number than in the candidates.csv file.
! wc -l /storage/dlwpt/annotations.csv

1187 /storage/dlwpt/annotations.csv


In [12]:
# The last column is also different
! head /storage/dlwpt/annotations.csv

seriesuid,coordX,coordY,coordZ,diameter_mm
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-128.6994211,-175.3192718,-298.3875064,5.651470635
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.7836509,-211.9251487,-227.12125,4.224708481
1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793540579077826395208,69.63901724,-140.9445859,876.3744957,5.786347814
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,-24.0138242,192.1024053,-391.0812764,8.143261683
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,2.441546798,172.4648812,-405.4937318,18.54514997
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,90.93171321,149.0272657,-426.5447146,18.20857028
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,89.54076865,196.4051593,-515.0733216,16.38127631
1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028192176989979435275,81.50964574,54.9572186,-150.3464233,10.36232088
1.3.6.1.4.1.14519.5.2.1.6279.6001.10268196240

We have size information for about 1,200 nodules. This is useful, since we can use it to make sure our training and validation data includes a representative spread of nodule sizes. Without this, it’s possible that our validation set could end up with only extreme values, making it seem as though our model is underperforming.



## Training and validation sets

We’ll split our data into training and validation sets. We want to make sure both sets are *representative* of the range of real-world input data we’re expecting to see and handle normally. If either set is meaningfully different from our real-world use cases, it’s pretty likely that our model will behave differently than we expect--all of the training and statistics we collect won’t be predictive once we transfer over to production use! We’re not trying to make this an exact science, but you should keep an eye out in future projects for hints that you are training and testing on data that doesn’t make sense for your operating environment.

Let’s get back to our nodules. We’re going to sort them by size and take every $N$th one for our validation set. That should give us the representative spread we’re looking for. Unfortunately, the location information provided in `annotations.csv` doesn’t always precisely line up with the coordinates in `candidates.csv`:

In [19]:
# These two coordinates are very close to each other.
! echo annotations.csv:
! grep 100225287222365663678666836860 /storage/dlwpt/annotations.csv
! echo candidates.csv:
! grep '100225287222365663678666836860.*,1$' /storage/dlwpt/candidates.csv 

annotations.csv:
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-128.6994211,-175.3192718,-298.3875064,5.651470635
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.7836509,-211.9251487,-227.12125,4.224708481
candidates.csv:
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,104.16480444,-211.685591018,-227.011363746,1
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-128.94,-175.04,-297.87,1


If we truncate the corresponding coordinates from each file, we end up with (-128.70, -175.32,-298.39) versus (-128.94,-175.04,-297.87). Since the nodule in question has a diameter of 5 mm, both of these points are clearly meant to be the “center” of the nodule, but they don’t line up exactly. It would be a perfectly valid response to decide that dealing with this data mismatch isn’t worth it, and to ignore the file. We are going to do the legwork to make things line up, though, since real-world datasets are often imperfect this way, and this is a good example of the kind of work you will need to do to assemble data from disparate data sources.